In [4]:
##CHROME SERVICE UPDATE##
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
# 
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.close()


In [5]:
##CHROMEDRIVER SERVICE CHECK##

from selenium import webdriver
import time
 
driver = webdriver.Chrome()
driver.get("https://google.com")
time.sleep(2)
driver.close()

In [9]:
from selenium.webdriver.common.by import By
##single hardcoded part - from query to printed url##
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Step 1: Open Google Scholar and search for the article
query = "article{duanlearning,title={Learning Coated Adversarial Camouflages for Object Detectors}, author={Duan, Yexin and Chen, Jialin and Zhou, Xingyu and Zou, Junhua and He, Zhengyun and Zhang, Jin and Zhang, Wu and Pan, Zhisong}}"
driver = webdriver.Chrome()
driver.get("https://scholar.google.com")
time.sleep(2) # Let the user actually see something!
search_box = driver.find_element(By.NAME,'q')
time.sleep(2) # Let the user actually see something!
search_box.send_keys(query)
time.sleep(2) # Let the user actually see something!
search_box.send_keys(Keys.RETURN)

# Step 2: Find the ".pdf" attribute in search results
pdf_link = driver.find_element(By.CSS_SELECTOR,'a[href*="/pdf/"]')     
#.find_elements_by_css_selector('a[href$=".pdf"]')

# Step 3: Print the found PDF links
print(pdf_link.get_attribute("href"))

# Close the browser
driver.quit()


https://arxiv.org/pdf/2109.00124


In [ ]:
## using the code iterating through the excel file##

##bug review/TBD:
# -couldnt find urls contains "/pdf/" - FIXED
# -files as above saved as non-pdf files (as default)
# -handle CAPTCHA by changing user-Agent or other leaps.

In [12]:
import random
from selenium.common import NoSuchElementException, StaleElementReferenceException
##First try - from table to download##

import time
import openpyxl
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Load the Excel file
excel_file_path = fr'C:\Users\Simon\Desktop\CBG\ReadingTaskAll.xlsx'
wb = openpyxl.load_workbook(excel_file_path)
sheet = wb.active

# Create an output directory for PDFs
output_dir = "PDFs"
os.makedirs(output_dir, exist_ok=True)

# Step 1: Open Google Scholar and search for each article
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
# options.add_argument("--headless")  # Run in headless mode
driver = webdriver.Chrome(options)

for row in sheet.iter_rows(min_row=2, values_only=True):
    # Introduce random delays
    delay = random.uniform(2, 7)  # Adjust the range as needed
    time.sleep(delay)
    bibtex = row[10]  # Assuming BibTeX is in column K
    driver.get("https://scholar.google.com")
    time.sleep(1)  # Let the user actually see something!
    search_box = driver.find_element(By.NAME,'q')
    search_box.send_keys(bibtex)
    print("~~~ROW 27 - send keys(bibtex)  DONE~~~")     ##4DEBUG
    # search_box.send_keys(Keys.RETURN)
    print("~~~ROW 29 - send keys(RETURN) - DONE~~~")    ##4DEBUG
    time.sleep(delay)  # Let the user actually see something!

    # Step 2: Find the ".pdf" attribute in search results
    try:
        pdf_link = driver.find_element(By.CSS_SELECTOR,'a[href$=".pdf"]')  
    except NoSuchElementException:
        try:
            print("~~~ROW 46 - inside 'try /pdf/'~~~")    ##4DEBUG
            pdf_link = driver.find_element(By.CSS_SELECTOR,'a[href*="/pdf/"]')     
        except NoSuchElementException:
            print("~~~ROW 49 - inside 'no such element exception, now 'continue' -'~~~")    ##4DEBUG
            continue  # Handle the case where no PDF link is found      
        
    if pdf_link:
        try:
            print(pdf_link.get_attribute("href"))
        except StaleElementReferenceException:
            print(" got StaleElementReferenceException")
            continue           ##couldnt find the file for "/pdf" (second row)
    else:
        print("No PDF link found")

    # Step 3: Download the PDFs and save them to the output directory
    pdf_url = pdf_link.get_attribute("href")
    response = requests.get(pdf_url)
    if response.status_code == 200:
        print("~~~ROW 52 - inside IF statment - DONE~~~")    ##4DEBUG

        # Get the file name from the URL and save it to the output directory
        pdf_filename = os.path.join(output_dir, os.path.basename(pdf_url))
        print("~~~ROW 56 - os.path.join - DONE~~~")    ##4DEBUG

        with open(pdf_filename, "wb") as pdf_file:
            pdf_file.write(response.content)
            print("~~~ROW 61 - Finished writing content - DONE~~~")    ##4DEBUG

# Close the browser
driver.quit()

# Close the Excel file
wb.close()


~~~ROW 27 - send keys(bibtex)  DONE~~~
~~~ROW 29 - send keys(RETURN) - DONE~~~
https://www.usenix.org/system/files/sec23summer_123-zhu-prepub.pdf
~~~ROW 52 - inside IF statment - DONE~~~
~~~ROW 56 - os.path.join - DONE~~~
~~~ROW 61 - Finished writing content - DONE~~~
~~~ROW 27 - send keys(bibtex)  DONE~~~
~~~ROW 29 - send keys(RETURN) - DONE~~~
~~~ROW 46 - inside 'try /pdf/'~~~
https://arxiv.org/pdf/2109.00124
~~~ROW 52 - inside IF statment - DONE~~~
~~~ROW 56 - os.path.join - DONE~~~
~~~ROW 61 - Finished writing content - DONE~~~
~~~ROW 27 - send keys(bibtex)  DONE~~~
~~~ROW 29 - send keys(RETURN) - DONE~~~
http://openaccess.thecvf.com/content/CVPR2022/papers/Zhu_Infrared_Invisible_Clothing_Hiding_From_Infrared_Detectors_at_Multiple_Angles_CVPR_2022_paper.pdf
~~~ROW 52 - inside IF statment - DONE~~~
~~~ROW 56 - os.path.join - DONE~~~
~~~ROW 61 - Finished writing content - DONE~~~
~~~ROW 27 - send keys(bibtex)  DONE~~~
~~~ROW 29 - send keys(RETURN) - DONE~~~
https://openaccess.thecvf.

KeyboardInterrupt: 

In [13]:
###previous WORKING CODE  - now cleaner (methods etc.)###
import random
from selenium.common import NoSuchElementException, StaleElementReferenceException
import time
import openpyxl
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

def set_up_browser():
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    # options.add_argument("--headless")  # Run in headless mode
    return webdriver.Chrome(options)

def search_article(driver, bibtex):
    driver.get("https://scholar.google.com")
    time.sleep(1)
    search_box = driver.find_element(By.NAME, 'q')
    search_box.send_keys(bibtex)
    time.sleep(random.uniform(2, 7))

def find_pdf_link(driver):
    try:
        pdf_link = driver.find_element(By.CSS_SELECTOR, 'a[href$=".pdf"]')
    except NoSuchElementException:
        try:
            pdf_link = driver.find_element(By.CSS_SELECTOR, 'a[href*="/pdf/"]')
        except NoSuchElementException:
            return None
    return pdf_link

def download_pdf(pdf_link, output_dir):
    pdf_url = pdf_link.get_attribute("href")
    response = requests.get(pdf_url)
    if response.status_code == 200:
        pdf_filename = os.path.join(output_dir, os.path.basename(pdf_url))
        with open(pdf_filename, "wb") as pdf_file:
            pdf_file.write(response.content)

def main():
    excel_file_path = fr'C:\Users\Simon\Desktop\CBG\ReadingTaskAll.xlsx'
    wb = openpyxl.load_workbook(excel_file_path)
    sheet = wb.active
    output_dir = "PDFs"
    os.makedirs(output_dir, exist_ok=True)

    driver = set_up_browser()

    for row in sheet.iter_rows(min_row=2, values_only=True):
        bibtex = row[10]

        search_article(driver, bibtex)
        pdf_link = find_pdf_link(driver)

        if pdf_link:
            print(pdf_link.get_attribute("href"))
            try:
                download_pdf(pdf_link, output_dir)
            except StaleElementReferenceException:
                print("StaleElementReferenceException occurred")

    driver.quit()
    wb.close()

if __name__ == "__main__":
    main()


https://www.usenix.org/system/files/sec23summer_123-zhu-prepub.pdf
https://arxiv.org/pdf/2109.00124
http://openaccess.thecvf.com/content/CVPR2022/papers/Zhu_Infrared_Invisible_Clothing_Hiding_From_Infrared_Detectors_at_Multiple_Angles_CVPR_2022_paper.pdf
https://openaccess.thecvf.com/content/CVPR2022/papers/Hu_Adversarial_Texture_for_Fooling_Person_Detectors_in_the_Physical_World_CVPR_2022_paper.pdf
http://openaccess.thecvf.com/content/CVPR2022/papers/Suryanto_DTA_Physical_Camouflage_Attacks_Using_Differentiable_Transformation_Network_CVPR_2022_paper.pdf


KeyboardInterrupt: 